In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from preprop import *
import os
import re

import emoji
from soynlp.normalizer import repeat_normalize

In [222]:
# https://github.com/Beomi/KcBERT
# !pip install soynlp emoji

In [223]:
data_dir = ['Unlabeled_data1', 'Unlabeled_data2', 'Unlabeled_data3']
dfall = []
for d in data_dir :
    d = 'unlabel_data/' + d
    files = [f for f in os.listdir(d) if f.endswith('xls') or f.endswith('xlsx')]
    for f in files :
        x1 = pd.ExcelFile( d + '/' + f)
#         print(x1.sheet_names)
        for sheet in x1.sheet_names :
            df = x1.parse(sheet)
            print(df.columns)
            print(df.shape)
            dfall.append(df)
            
df = pd.concat(dfall)      
print(df.shape)
df.reset_index(drop=True, inplace=True)
df.to_csv('files/unlabel.csv', index=None)


Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(38557, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(56346, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(35266, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(11965, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(62317, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(64478, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(39577, 13)

In [2]:
# df.to_csv('files/unlabel.csv', index=None)
df = pd.read_csv('files/unlabel.csv')
df = df[['진단명', 'SE']]
df.shape

(981339, 2)

In [3]:
# data_dir = ['PMS_unlabel_2차분']
# dfall = []
# for d in data_dir :
#     d = 'unlabel_data/' + d
#     files = [f for f in os.listdir(d) if f.endswith('xls') or f.endswith('xlsx')]
#     for f in files :
#         x1 = pd.ExcelFile( d + '/' + f)
# #         print(x1.sheet_names)
#         for sheet in x1.sheet_names :
#             df = x1.parse(sheet)
#             print(df.columns)
#             print(df.shape)
#             dfall.append(df)
            
# df = pd.concat(dfall)      
# print(df.shape)
# df.reset_index(drop=True, inplace=True)
# # df.to_csv('files/unlabel.csv', index=None)

In [4]:
x2 = pd.ExcelFile('unlabel_data/PMS_unlabel_data.xlsx')
for sheet in x2.sheet_names :
    print(sheet)
    df1 = x2.parse(sheet)
    print(df1.columns)
df1 = x2.parse('Sheet2')


Sheet2
Index(['Sheet 1', 'ASSESSMENT_NAME', 'CHART_ID', 'CHART_MEMO', 'CHART_DATE',
       'COMPANY_NAME'],
      dtype='object')
Sheet1
Index([], dtype='object')


In [5]:
df1 = df1[['ASSESSMENT_NAME', 'CHART_MEMO']]
df1.columns = ['진단명', 'SE']
df1.shape

(106750, 2)

In [6]:
df = pd.concat([df, df1])
df.reset_index(drop=True, inplace=True)
df.shape

(1088089, 2)

In [7]:
df['진단명'].value_counts()[0:50]

Pancreatitis                                   4464
RESPIRATORY DISTRESS SYNDROME, ACUTE           3194
Otitis Externa                                 3149
Anti-inflamation                               2844
Malassezia Dermatitis                          2374
Vomiting, Acute                                2244
Pulmonary Edema                                2104
Dyspnea and Tachypnea                          2012
Chronic Renal Disease (CKD) - IRIS stage       1947
dermatits-bacterial                            1846
Dermatophytosis                                1713
Blood Typing: DEA 1.1 positive                 1655
Vaccination                                    1431
Cancer PS(Prevalence Survey)                   1391
OTITIS EXTERNA - BACTERIAL INFECTION           1384
Dressing                                       1377
췌장염(Pancreatitis)                              1323
Ear flushing                                   1253
Mitral valve insufficiency (MVI)               1223
Anorexia    

In [9]:
df['진단명'].value_counts()[0:1000].sum()

210885

In [10]:
df.dropna(subset=['SE'], inplace=True)
df.shape

(1065829, 2)

In [11]:
df.drop_duplicates(subset=['SE'], inplace=True)
df.shape

(872491, 2)

In [12]:
df['len'] = df['SE'].apply(lambda x : len(str(x)))
df = df[df['len'] > 50]
df.shape

(693898, 3)

In [13]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [14]:
def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [15]:
df.SE = df.SE.apply(lambda x : clean(str(x)))

In [16]:
df['len'] = df['SE'].apply(lambda x : len(str(x)))
df = df[df['len'] > 50]
df.shape

(674445, 3)

In [18]:
def clean1(text) :
    text = phone_number_filter(text)
    text = url_filter(text)
    text = price_filter(text)
    text = date_filter(text)
    text = time_filter(text)
    text = remove_name(text)
    text = re.sub('[ ][0-9]+[ ]','', text )
    text = re.sub('[\s]+[a-z][\s]+', ' ', text )
    text = re.sub('[.]+', '.', text )
    text = re.sub('[-]+', '-', text )
    text = re.sub('[ ]+', ' ', text )    
    return text

In [19]:
# df.SE = df.SE.apply(lambda x : preprocess(str(x)))
df_pre1 = df.copy()
df.SE = df.SE.apply(lambda x : clean1(str(x)))
df_pre1.SE = df_pre1.SE.apply(lambda x : preprocess(str(x)))
df['len'] = df['SE'].apply(lambda x : len(str(x)))
df_pre1['len'] = df_pre1['SE'].apply(lambda x : len(str(x)))

In [20]:
df = df[df['len'] > 50]
df.shape

(672172, 3)

In [21]:
df_pre1 = df_pre1[df_pre1['len'] > 50]
df_pre1.shape

(659565, 3)

In [22]:
df.head()

,진단명,SE,len
0,Anorexia,"[ ] - 체온 37.5 - 호흡수 36-24-27, shivering 있음 - M...",463
1,Anorexia,1.CC : 식욕부진 2.HPI - 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계...,765
16,NaN,"[퇴원기록 by 최새롬] - 식욕부진으로 내원, 내원 후 보호자분 비용부담으로 최소...",318
17,NaN,"[S] - 설사 지속, 계속 끙끙거리면서 잠을 못 이룸 - 오늘 아침부터 호흡 및 ...",219
18,Splenic mass,1.CC : 복강 내 종양 및 담낭 파열 의심 (서울 예스ah) 2.HPI - mo...,1086


In [23]:
df_pre1.head()

,진단명,SE,len
0,Anorexia,체온 호흡수 shivering 있음 mm p.pink skin turgor 약간 ...,337
1,Anorexia,cc 식욕부진 hpi 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계속 한달 전부터...,584
16,NaN,퇴원기록 by 최새롬 식욕부진으로 내원 내원 후 보호자분 비용부담으로 최소한의 스...,293
17,NaN,설사 지속 계속 끙끙거리면서 잠을 못 이룸 오늘 아침부터 호흡 및 기력 안 좋아짐...,193
18,Splenic mass,cc 복강 내 종양 및 담낭 파열 의심 서울 예스ah hpi month 전 부터 연...,873


In [253]:
df.describe()

,len
count,698619.000000
mean,268.324764
std,296.026361
min,41.000000
25%,94.000000
50%,174.000000
75%,333.000000
max,25356.000000


In [199]:
# dft = df['SE']
# dft.to_csv('files/pet.txt', index=None, header=False) 

In [156]:
# from tokenizers import ByteLevelBPETokenizer

# tokenizer = ByteLevelBPETokenizer()

# tokenizer.train(files='files/pet.txt', vocab_size=32000, min_frequency=50, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])

In [178]:
# tokenizer.encode('hi my name is ,, 안녕 나는 고양이 좋아해').tokens

In [162]:
# df.len = df.SE.apply(lambda x : len(tokenizer.encode(str(x)).tokens))

In [254]:
df.head()

,진단명,SE,len
0,Anorexia,체온 호흡수 shivering 있음 mm p.pink skin turgor 약간 ...,337
1,Anorexia,cc 식욕부진 hpi 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계속 한달 전부터...,584
16,NaN,퇴원기록 by 최새롬 식욕부진으로 내원 내원 후 보호자분 비용부담으로 최소한의 스...,293
17,NaN,설사 지속 계속 끙끙거리면서 잠을 못 이룸 오늘 아침부터 호흡 및 기력 안 좋아짐...,193
18,Splenic mass,cc 복강 내 종양 및 담낭 파열 의심 서울 예스ah hpi month 전 부터 연...,873


In [24]:
df1 = df_pre1.copy()
df1 = df1[['진단명','SE'] ]
df1.head()

,진단명,SE
0,Anorexia,체온 호흡수 shivering 있음 mm p.pink skin turgor 약간 ...
1,Anorexia,cc 식욕부진 hpi 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계속 한달 전부터...
16,NaN,퇴원기록 by 최새롬 식욕부진으로 내원 내원 후 보호자분 비용부담으로 최소한의 스...
17,NaN,설사 지속 계속 끙끙거리면서 잠을 못 이룸 오늘 아침부터 호흡 및 기력 안 좋아짐...
18,Splenic mass,cc 복강 내 종양 및 담낭 파열 의심 서울 예스ah hpi month 전 부터 연...


In [25]:
df1['SE'] = df1[df1.columns[:]].apply(
            lambda x : ','.join(x.dropna().astype(str)),
            axis=1)

In [26]:
df1.head()

,진단명,SE
0,Anorexia,"Anorexia, 체온 호흡수 shivering 있음 mm p.pink skin t..."
1,Anorexia,"Anorexia,cc 식욕부진 hpi 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 ..."
16,NaN,퇴원기록 by 최새롬 식욕부진으로 내원 내원 후 보호자분 비용부담으로 최소한의 스...
17,NaN,설사 지속 계속 끙끙거리면서 잠을 못 이룸 오늘 아침부터 호흡 및 기력 안 좋아짐...
18,Splenic mass,"Splenic mass,cc 복강 내 종양 및 담낭 파열 의심 서울 예스ah hpi..."


In [30]:
df = df[['진단명','SE'] ]
df.head()

,진단명,SE
0,Anorexia,"[ ] - 체온 37.5 - 호흡수 36-24-27, shivering 있음 - M..."
1,Anorexia,1.CC : 식욕부진 2.HPI - 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계...
16,NaN,"[퇴원기록 by 최새롬] - 식욕부진으로 내원, 내원 후 보호자분 비용부담으로 최소..."
17,NaN,"[S] - 설사 지속, 계속 끙끙거리면서 잠을 못 이룸 - 오늘 아침부터 호흡 및 ..."
18,Splenic mass,1.CC : 복강 내 종양 및 담낭 파열 의심 (서울 예스ah) 2.HPI - mo...


In [31]:
print(df.shape)
print(df1.shape)

(672172, 2)
(659565, 2)


In [32]:
dfa = pd.concat([df, df1])
dfa.reset_index(drop=True, inplace=True)
dfa.shape

(1331737, 2)

In [33]:
dfa.head()

,진단명,SE
0,Anorexia,"[ ] - 체온 37.5 - 호흡수 36-24-27, shivering 있음 - M..."
1,Anorexia,1.CC : 식욕부진 2.HPI - 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계...
2,NaN,"[퇴원기록 by 최새롬] - 식욕부진으로 내원, 내원 후 보호자분 비용부담으로 최소..."
3,NaN,"[S] - 설사 지속, 계속 끙끙거리면서 잠을 못 이룸 - 오늘 아침부터 호흡 및 ..."
4,Splenic mass,1.CC : 복강 내 종양 및 담낭 파열 의심 (서울 예스ah) 2.HPI - mo...


In [37]:
dfa = dfa.sample(frac=1)
dfa.head()

,진단명,SE
1282558,교상,"교상,radiographic findings thorax vhs approx. mi..."
1113909,NaN,요새 들어서 밥 좀 잘 안 먹음 치우려고 하면 다 먹음 소량씩 급여 후 time 정...
796998,NaN,퇴원기록 by 이혜경 김진경선생님에게 인계 입원 경과 primary imt 입원치...
174464,NaN,"month 전부터 귀 안좋아짐. 양측 귀 발적, 부종. 귀털 뽑고 귀드레싱. 주사..."
1097141,NaN,오전관리사항 food 금식 urine feces etc tx cefa enro r...


In [38]:
# dfa.head(20)

In [39]:
dft = dfa['SE']
dft.to_csv('files/pet_0814.txt', index=None, header=False) 